In [1]:
import pandas as pd
import numpy as np
import csv
import utility

In [2]:
#df=pd.read_csv('dataset/train-queries.csv',sep=";")

In [3]:
#len(df.columns)

In [4]:
"""print("number of records",len(df))
print("number of unique users",len(df['userId'].unique()))
print("number of rows with missing userId",sum(df['userId'].isnull()))"""

'print("number of records",len(df))\nprint("number of unique users",len(df[\'userId\'].unique()))\nprint("number of rows with missing userId",sum(df[\'userId\'].isnull()))'

In [5]:
"""df_test=df.loc[df['is.test']==True]
df_train=df.loc[df['is.test']==False]
df_test.to_csv('dataset/split_test_queries.csv',sep=';',index=False)
df_train.to_csv('dataset/split_train_queries.csv',sep=';',index=False)"""

"df_test=df.loc[df['is.test']==True]\ndf_train=df.loc[df['is.test']==False]\ndf_test.to_csv('dataset/split_test_queries.csv',sep=';',index=False)\ndf_train.to_csv('dataset/split_train_queries.csv',sep=';',index=False)"

In [6]:
"""print("number of train queries",len(df_train))
print("number of test queries",len(df_test))
len(df_train.columns)"""

'print("number of train queries",len(df_train))\nprint("number of test queries",len(df_test))\nlen(df_train.columns)'

In [7]:
df_train_query=pd.read_csv('dataset-train/split_train_queries.csv',sep=";")

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
categry=pd.read_csv('dataset-train/product-categories.csv',sep=';')

In [9]:
print categry.columns
print len(categry['categoryId'].unique())

Index([u'itemId', u'categoryId'], dtype='object')
1217


In [21]:
df_train_query.head()

,queryId,sessionId,userId,timeframe,duration,eventdate,searchstring.tokens,categoryId,items,is.test
0,1,1,NaN,16327074,311,2016-05-09,"16655,244087,51531,529597,58153",0,"7518,71,30311,7837,30792,8252,81766,9338,62220...",False
1,2,2,NaN,705527,314,2016-05-09,"528941,529116",0,"70095,15964,8627,134850,32754,100747,74771,314...",False
2,4,4,NaN,0,1092,2016-05-09,"3918,3822,460416,528812,5276,529517,528738",0,"46632,57465,79064,57748,6080,35997,47088,6078,...",False
3,5,5,NaN,102700,266,2016-05-09,"529223,199482",0,"27312,84626,12621,46209,5140,57539,5368,12923,...",False
4,6,6,NaN,0,673,2016-05-09,75084,0,"20519,59225,26264,73191,73331,40988,2165,43303...",False


In [113]:
#check to see whether a query was targeting towards a product category
#l=list(str(df_train_query[df_train_query['queryId']==1]['items'][0]).split(','))
dft=df_train_query[df_train_query['queryId']==76]['items']
for p in dft.all().split(','):
    print categry[categry['itemId']==int(p)]#.categryId#['categoryId']


        itemId  categoryId
163562     383         299
        itemId  categoryId
163598   49094         299
        itemId  categoryId
163599  110037         299
        itemId  categoryId
163652   40921         299
        itemId  categoryId
163666   98924         299
        itemId  categoryId
163949    2641         774
        itemId  categoryId
163678   99714         299
        itemId  categoryId
163509   36055         299
        itemId  categoryId
163520   47616         299
        itemId  categoryId
163522   90603         299
        itemId  categoryId
163531   14912         299
        itemId  categoryId
163954   10989         774
        itemId  categoryId
163735   11634         299
        itemId  categoryId
163773   78103         299
        itemId  categoryId
163778   56627         299
        itemId  categoryId
163805   35786         299
        itemId  categoryId
163815   19581         299
        itemId  categoryId
163967    6237         774
        itemId  categoryId
1

In [8]:
df_train_query.head()

,queryId,sessionId,userId,timeframe,duration,eventdate,searchstring.tokens,categoryId,items,is.test
0,1,1,NaN,16327074,311,2016-05-09,"16655,244087,51531,529597,58153",0,"7518,71,30311,7837,30792,8252,81766,9338,62220...",False
1,2,2,NaN,705527,314,2016-05-09,"528941,529116",0,"70095,15964,8627,134850,32754,100747,74771,314...",False
2,4,4,NaN,0,1092,2016-05-09,"3918,3822,460416,528812,5276,529517,528738",0,"46632,57465,79064,57748,6080,35997,47088,6078,...",False
3,5,5,NaN,102700,266,2016-05-09,"529223,199482",0,"27312,84626,12621,46209,5140,57539,5368,12923,...",False
4,6,6,NaN,0,673,2016-05-09,75084,0,"20519,59225,26264,73191,73331,40988,2165,43303...",False


In [9]:
print("number of unique queries",len(df_train_query['queryId'].unique()))
print("number of unique sessions",len(df_train_query['sessionId'].unique()))
print("number of unique users",len(df_train_query['userId'].unique()))
print("number of unique search strings",len(df_train_query['searchstring.tokens'].unique()))

('number of unique queries', 636160)
('number of unique sessions', 368782)
('number of unique users', 140388)
('number of unique search strings', 18416)


In [83]:
# Loading item views; taking itemId column
item_views = pd.read_csv('dataset-train/train-item-views.csv', sep=';')
print('Item views', len(item_views))

# Loading clicks; taking itemId column
clicks = pd.read_csv('dataset-train/train-clicks_with_usrId.csv', sep=';')
print('Clicks', len(clicks))

# Loading purchases; taking itemId column
purchases = pd.read_csv('dataset-train/train-purchases.csv', sep=';')
print('Purchases', len(purchases))

('Item views', 1235380)
('Clicks', 1127764)
('Purchases', 18025)


In [75]:
#l=clicks.merge(df_train_query[['queryId','userId']], on='queryId', how='inner')['userId']

In [77]:
#clicks['userId']=l

In [84]:
#clicks.head()

In [85]:
#df_train_query[df_train_query['queryId']==1]['items'][0]

In [80]:
#clicks.to_csv('dataset-train/train-clicks_with_usrId.csv',sep=';',index=False)

In [81]:
#clicks_=pd.read_csv('dataset-train/train-clicks_with_usrId.csv',sep=';')

In [86]:
users=df_train_query['userId'].unique()

In [112]:
print item_views.columns
print clicks.columns
print purchases.columns
print df_train_query.columns

Index([u'sessionId', u'userId', u'itemId', u'timeframe', u'eventdate'], dtype='object')
Index([u'queryId', u'timeframe', u'itemId', u'userId'], dtype='object')
Index([u'sessionId', u'userId', u'timeframe', u'eventdate', u'ordernumber',
       u'itemId'],
      dtype='object')
Index([u'queryId', u'sessionId', u'userId', u'timeframe', u'duration',
       u'eventdate', u'searchstring.tokens', u'categoryId', u'items',
       u'is.test'],
      dtype='object')


In [16]:
len(users)

140388

In [35]:
user_product_pref={}
i=1
#for user in users:
for user in [6.0,10.0]:
    print "\ruser "+str(i)+":"+str(user),
    prod_pref={}
    #df_temp=df_train_query[df_train_query['userId']==user]
    user_item_views=item_views[item_views['userId']==user]['itemId']
    user_purchases=purchases[purchases['userId']==user]['itemId']
    user_clicks=clicks[clicks['userId']==user]['itemId']
    #for ii,row in df_temp.iterrows():        
    for cost, container in enumerate([user_item_views, user_clicks, user_purchases]):
        for prod in container.values:
            product = str(prod)
            print(user,product,cost)
            if product not in prod_pref:
                prod_pref[product] = cost
            else:
                prod_pref[product] += cost
    user_product_pref[user]=prod_pref
    i+=1

'user_product_cost={}\ni=1\n#for user in users:\nfor user in [6.0,10.0]:\n    print "\ruser "+str(i)+":"+str(user),\n    prod_pref={}\n    df_temp=df_train_query[df_train_query[\'userId\']==user]\n    user_item_views=item_views[item_views[\'userId\']==user][\'itemId\']\n    user_purchases=purchases[purchases[\'userId\']==user][\'itemId\']\n    for ii,row in df_temp.iterrows():        \n        user_clicks=clicks[clicks[\'queryId\']==row[\'queryId\']][\'itemId\']\n        for cost, container in enumerate([user_item_views, user_clicks, user_purchases]):\n            for prod in container.values:\n                product = str(prod)\n                print(user,product,cost)\n                if product not in prod_pref:\n                    prod_pref[product] = cost\n                else:\n                    prod_pref[product] += cost\n    user_product_cost[user]=prod_pref\n    i+=1'

In [94]:
#save_object(user_product_cost,'user_product_cost.pkl')
user_product_cost=load_object('user_product_cost.pkl')

In [110]:
#df_train_query.head(100)
#print user_product_cost[6.0]
print user_product_cost[2110.0]


{'3717': 1}


In [132]:
#item_views[item_views['userId']==10.0]
item_views[item_views['userId']==6.0]

,sessionId,userId,itemId,timeframe,eventdate
535,129,6.0,7837,3966,2016-03-22
45706,18349,6.0,98888,941251,2016-03-09
45707,18349,6.0,180773,14952,2016-03-09
612330,210228,6.0,1009,41101,2016-04-10
612331,210228,6.0,21344,308440,2016-04-10
612332,210228,6.0,111668,247919,2016-04-10
612333,210228,6.0,10890,127364,2016-04-10
650695,226022,6.0,107654,268775,2016-04-27
650696,226022,6.0,60650,103601,2016-04-27
650697,226022,6.0,75568,52474,2016-04-27


In [133]:
clicks[clicks['userId']==6.0]

,queryId,timeframe,itemId,userId
452,130,3113,7837,6.0
32973,22506,14181,180773,6.0
611810,259698,40006,1009,6.0
611811,259698,306891,21344,6.0
611812,259698,246122,111668,6.0
611813,259698,125867,10890,6.0
646247,282304,102842,60650,6.0
646248,282304,51117,75568,6.0


In [ ]:
#df_views=pd.read_csv('dataset/train-item-views.csv',sep=';')

In [ ]:
#df_views.loc[df_views['sessionId']==16]
#df_train.loc[df_train['sessionId']==2]

In [ ]:
#df_views.loc[df_views['sessionId']==2]

In [ ]:
#df_test=pd.read_csv('dataset/split_test_queries.csv',sep=';')

In [ ]:
#df_test.head()